## Synopsis

This script takes the averaged HDF files on `/net/wolf` with neat circular mask for SAXS, and replaces the SAXS 1D with the calculated absolute cross-section values.


## Initialization

In [1]:

import numpy as np
from numpy import sqrt, sin, cos, pi, exp
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import h5py  
from subprocess import PIPE, run
from xpcs_viewer import XpcsFile as xf


## Transmission coefficient calculation

In [2]:

pind4_flux = 1.31e10 # Photon per second
f = 2e-3 # 2 mm capillary
t_exp = 19e-6 # Exposure time per frame
Del_Omega = (76e-6/8)**2
pind4_base = 147

# The 0.01 coefficient is to convert the absolute cross-section unit from m to cm
coeff_1 = 1/(t_exp*f*Del_Omega*pind4_flux)*0.01

#Scan 115 
pind4_full_1 = 15087
pind4_cap = 12920
pind4_water = 5833

Tr_water = (pind4_water-pind4_base)/(pind4_full_1-pind4_base)
Tr_cap = (pind4_cap-pind4_base)/(pind4_full_1-pind4_base)

# Scan 128
pind4_full_2 = 15419
pind4_NaCl = 5394
pind4_NH4 = 4796

Tr_NaCl = (pind4_NaCl-pind4_base)/(pind4_full_2-pind4_base)
Tr_NH4 = (pind4_NH4-pind4_base)/(pind4_full_2-pind4_base)


## Read background `saxs_1d`

In [3]:

fp_remote = '/net/wolf/data/xpcs8/2020-3/qzhang202010/cluster_results_SAXS/'

a = xf('K063/AvgK063_00001_Empty_Cap_att00_Try1_0001-100000.hdf', cwd = fp_remote)
Iq_cap = a.saxs_1d['Iq']


## List target file names 

In [4]:

fn_remote = ('J058/AvgJ058_00001_CPMV_6C_7v1_Water_att00_Try1_0001-100000.hdf',
             'M071/AvgM071_00001_CPMV_6C_7v1_NH4M_att00_Try1_0001-100000.hdf',
             'L067/AvgL067_00001_CPMV_6C_7v1_NaCl4M_att00_Try1_0001-100000.hdf')
                                    
fn_local = ('J058_Water_SAXS_Abs_Cross.hdf', 
            'M071_NH4_SAXS_Abs_Cross.hdf', 
            'L067_NaCl_SAXS_Abs_Cross.hdf')
             
tr_coeff = (Tr_water, 
            Tr_NH4, 
            Tr_NaCl)             
                

In [5]:

for ii in range(3):
    
    a = xf(fn_remote[ii], cwd = fp_remote)
    
    command = ['cp', fp_remote+fn_remote[ii], fn_local[ii]]    
    _ = run(command, stdout=PIPE, stderr=PIPE)
    
    with h5py.File(fn_local[ii], 'r+') as f1:
        f1['/exchange/partition-mean-total'][...] = \
            (a.saxs_1d['Iq']/tr_coeff[ii]-Iq_cap/Tr_cap)*coeff_1
        f1.flush()
                   